In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import MiniBatchKMeans
from sklearn.mixture import GaussianMixture
from sklearn.cluster import AgglomerativeClustering
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
import pandas as pd

In [2]:
data = pd.read_csv("2024pitches.csv")
pd.set_option('display.max_columns', None)
data.head(20)

,Unnamed: 0,index,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,fielder_2,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,estimated_slg_using_speedangle,delta_pitcher_run_exp,hyper_speed,home_score_diff,bat_score_diff,home_win_exp,bat_win_exp,age_pit_legacy,age_bat_legacy,age_pit,age_bat,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle
0,0,1669,NaN,2024-03-16,NaN,NaN,NaN,"Jackson, Gabriel",687637,682791,single,hit_into_play,NaN,NaN,NaN,NaN,NaN,Dylan Beavers singles on a soft ground ball to...,S,L,R,BAL,BOS,X,1.0,ground_ball,1,0,2024,NaN,NaN,NaN,NaN,682983.0,642021.0,NaN,2,9,Bot,118.13,186.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,747948,656627,694359,642197,656388,686765,681468,694812,676628,NaN,NaN,NaN,0.9,NaN,1.0,0.0,NaN,75,2,NaN,4,4,4,4,4,5,5,4,NaN,NaN,NaN,0.370,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.630,0.630,22,22,23,23,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1736,NaN,2024-03-16,NaN,NaN,NaN,"Jackson, Gabriel",687637,682791,NaN,ball,NaN,NaN,NaN,NaN,NaN,Dylan Beavers singles on a soft ground ball to...,S,L,R,BAL,BOS,B,NaN,NaN,0,0,2024,NaN,NaN,NaN,NaN,682983.0,NaN,642021.0,2,9,Bot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,747948,656627,694359,642197,656388,686765,681468,694812,676628,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75,1,NaN,4,4,4,4,4,4,4,4,NaN,NaN,NaN,-0.005,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.635,0.635,22,22,23,23,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1880,NaN,2024-03-16,NaN,NaN,NaN,"Jackson, Gabriel",642021,682791,walk,ball,NaN,NaN,NaN,NaN,NaN,Errol Robinson walks.,S,R,R,BAL,BOS,B,NaN,NaN,3,0,2024,NaN,NaN,NaN,NaN,682983.0,NaN,NaN,2,9,Bot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,747948,656627,694359,642197,656388,686765,681468,694812,676628,NaN,NaN,NaN,0.7,NaN,0.0,0.0,NaN,74,4,NaN,4,4,4,4,4,4,4,4,NaN,NaN,NaN,0.007,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.628,0.628,22,29,23,30,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2002,NaN,2024-03-16,NaN,NaN,NaN,"Jackson, Gabriel",642021,682791,NaN,ball,NaN,NaN,NaN,NaN,NaN,Errol Robinson walks.,S,R,R,BAL,BOS,B,NaN,NaN,2,0,2024,NaN,NaN,NaN,NaN,682983.0,NaN,NaN,2,9,Bot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,747948,656627,694359,642197,656388,686765,681468,694812,676628,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74,3,NaN,4,4,4,4,4,4,4,4,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.628,0.628,22,29,23,30,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2156,NaN,2024-03-16,NaN,NaN,NaN,"Jackson, Gabriel",642021,682791,NaN,ball,NaN,NaN,NaN,NaN,NaN,Errol Robinson walks.,S,R,R,BAL,BOS,B,NaN,NaN,1,0,2024,NaN,NaN,NaN,NaN,682983.0,NaN,NaN,2,9,Bot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,747948,656627,694359,642197,656388,686765,681468,694812,676628,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74,2,NaN,4,4,4,4,4,4,4,4,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0.628

In [3]:
data.shape

(757713, 115)

In [4]:
columns = [
    "pitch_type", "release_speed", "release_pos_x", "release_pos_z", "batter", "pitcher", "events", "description", "zone", "des",
    "game_type", "stand", "p_throws", "type", "balls", "strikes", "pfx_x", "pfx_z", "plate_x", "plate_z", "on_3b", "on_2b", "on_1b",
    "hc_x", "hc_y", "sv_id", "vx0", "vy0", "vz0", "ax", "ay", "az", "sz_top", "sz_bot", "hit_distance_sc", "launch_speed",
    "launch_angle", "effective_speed", "release_spin_rate", "release_extension", "game_pk", "release_pos_y",
    "estimated_woba_using_speedangle", "launch_speed_angle", "at_bat_number", "pitch_number", "pitch_name", "home_score",
    "away_score", "spin_axis", "delta_home_win_exp", "delta_run_exp", "bat_speed", "swing_length",
    "estimated_slg_using_speedangle", "delta_pitcher_run_exp", "bat_win_exp", "n_thruorder_pitcher",
    "n_priorpa_thisgame_player_at_bat", "arm_angle"]

pitch_data = data[columns]

In [5]:
pd.set_option('display.max_rows', None)
pitch_data.isnull().sum(axis=0)

pitch_type                           13686
release_speed                        14889
release_pos_x                        14888
release_pos_z                        14888
batter                                   0
pitcher                                  0
events                              560351
description                              0
zone                                 14888
des                                      0
game_type                                0
stand                                    0
p_throws                                 0
type                                     0
balls                                    0
strikes                                  0
pfx_x                                14888
pfx_z                                14888
plate_x                              14888
plate_z                              14888
on_3b                               686878
on_2b                               616465
on_1b                               527641
hc_x       

In [6]:
#pitch_data.to_csv('pitchdata.csv', index=False)

In [7]:
features = ["release_speed", "release_pos_x", "release_pos_z", "release_spin_rate", "pfx_x", 
            "pfx_z", "vx0", "vy0", "vz0", "ax", "ay", "az", "release_extension", "spin_axis"]

pitch_data_filtered = pitch_data.dropna(subset=features).copy()
pitch_data_filtered.loc[:, features] = pitch_data_filtered[features].apply(pd.to_numeric, errors="coerce")

scaler = StandardScaler()
pitch_data_filtered.loc[:, features] = scaler.fit_transform(pitch_data_filtered[features])
pitch_data_filtered = pitch_data_filtered.sample(frac=1/2, random_state=4400) #take sample to speed up clustering

pitch_data_filtered["cluster_label"] = -1

cluster_range = range(2, 9)
clustering_results = []
total_pitches = pitch_data_filtered["pitch_type"].nunique()



In [8]:
for i, pitch in enumerate(pitch_data_filtered["pitch_type"].unique()):
    subset = pitch_data_filtered[pitch_data_filtered["pitch_type"] == pitch].copy()

    if len(subset) < 500:  # Skip pitch types with very few data points
        continue

    print(f"Processing {pitch} ({i+1}/{total_pitches})")  # Track progress

    inertia = []
    silhouette_scores = []
    davies_bouldin_scores = []

    for k in cluster_range:
        kmeans = MiniBatchKMeans(n_clusters=k, random_state=4400, batch_size=1000, n_init=10)
        labels = kmeans.fit_predict(subset[features])

        inertia.append(kmeans.inertia_)
        silhouette_avg = silhouette_score(subset[features], labels)
        silhouette_scores.append(silhouette_avg)
        db_index = davies_bouldin_score(subset[features], labels)
        davies_bouldin_scores.append(db_index)

    # Store results
    for j, k in enumerate(cluster_range):
        clustering_results.append({
            "Pitch Type": pitch,
            "Clusters": k,
            "Inertia": inertia[j],
            "Silhouette Score": silhouette_scores[j],
            "Davies-Bouldin Index": davies_bouldin_scores[j]})

clustering_results_df = pd.DataFrame(clustering_results)

Processing FF (1/17)
Processing SI (2/17)
Processing CH (3/17)
Processing CU (4/17)
Processing FC (5/17)
Processing ST (6/17)
Processing KC (7/17)
Processing SL (8/17)
Processing FS (9/17)
Processing SV (14/17)


In [9]:
clustering_results_df

,Pitch Type,Clusters,Inertia,Silhouette Score,Davies-Bouldin Index
0,FF,2,543355.294672,0.401189,1.053099
1,FF,3,471748.347774,0.217551,1.646960
2,FF,4,444447.142545,0.183969,1.827537
3,FF,5,406188.049671,0.151949,1.919827
4,FF,6,400799.227071,0.172301,1.715706
5,FF,7,369372.188728,0.134685,1.821111
6,FF,8,358260.187371,0.132036,1.909219
7,SI,2,358150.939034,0.510988,0.812243
8,SI,3,303713.795005,0.300096,1.347992
9,SI,4,276167.883396,0.229149,1.614944


In [10]:
manual_cluster_counts = {
    "FF": 3,  # Four-seam
    "SI": 3,  # Sinker
    "CH": 3,  # Changeup
    "CU": 3,  # Curveball
    "FC": 2,  # Cutter
    "ST": 2,  # Sweeper
    "KC": 4,  # Knuckle Curve
    "SL": 2,  # Slider
    "FS": 3,  # Split/Fork
    "SV": 3   # Slurve
}

for pitch in pitch_data_filtered["pitch_type"].unique():
    subset = pitch_data_filtered[pitch_data_filtered["pitch_type"] == pitch].copy()

    if len(subset) < 10:  
        continue
    k = manual_cluster_counts.get(pitch, 3)  

    kmeans = KMeans(n_clusters=k, random_state=4400, n_init=10)
    subset["cluster_label"] = kmeans.fit_predict(subset[features])

    # Merge results back into main dataframe
    pitch_data_filtered.loc[subset.index, "cluster_label"] = subset["cluster_label"]

In [11]:
pitch_data_filtered.head()

,pitch_type,release_speed,release_pos_x,release_pos_z,batter,pitcher,events,description,zone,des,game_type,stand,p_throws,type,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,hc_x,hc_y,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_woba_using_speedangle,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,estimated_slg_using_speedangle,delta_pitcher_run_exp,bat_win_exp,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,arm_angle,cluster_label
573345,FF,0.574718,-0.787149,0.020814,669016,663623,NaN,ball,11.0,Brandon Marsh strikes out swinging.,R,L,R,B,0,1,-0.957719,0.809270,-0.58,4.03,NaN,664761.0,592206.0,NaN,NaN,NaN,0.700710,-0.580923,0.877887,-1.020710,1.015764,0.695777,3.10,1.35,NaN,NaN,NaN,93.1,0.246067,0.754817,745539,53.74,NaN,NaN,10,2,4-Seam Fastball,1,0,0.718123,0.0,0.053,NaN,NaN,NaN,-0.053,0.645,1,0,26.0,0
368247,FF,1.261108,2.326286,-0.397030,669208,640462,NaN,ball,11.0,Ryan Bliss called out on strikes.,R,R,L,B,0,1,1.248267,0.795096,-0.03,3.62,NaN,NaN,NaN,NaN,NaN,NaN,-2.459973,-1.223750,0.512231,1.663833,1.109399,0.867896,3.15,1.42,NaN,NaN,NaN,98.0,0.008366,1.629836,746051,53.32,NaN,NaN,37,2,4-Seam Fastball,6,2,-0.656916,0.0,0.023,NaN,NaN,NaN,-0.023,0.095,1,1,26.0,2
68467,SI,0.591459,-0.044838,0.628586,596117,668933,NaN,ball,11.0,Garrett Stubbs walks.,R,L,R,B,0,0,-0.623479,-0.026946,-1.44,2.56,NaN,NaN,NaN,NaN,NaN,NaN,-0.398000,-0.608688,-0.236596,-0.556328,0.700154,-0.034586,2.91,1.24,NaN,NaN,NaN,91.3,-0.439397,-2.088995,745598,55.04,NaN,NaN,37,1,Sinker,2,1,0.566869,0.0,0.015,NaN,NaN,NaN,-0.015,0.677,2,1,42.0,1
195425,FF,0.038998,-0.375941,1.882117,502054,663474,NaN,ball,12.0,Tommy Pham walks.,R,R,R,B,0,1,0.011578,1.489581,1.06,5.22,NaN,NaN,NaN,NaN,NaN,NaN,0.719435,-0.045117,0.839569,-0.039520,0.101317,1.225858,3.43,1.57,NaN,NaN,NaN,90.0,0.149328,0.536062,746797,53.81,NaN,NaN,4,2,4-Seam Fastball,0,0,0.113106,0.0,0.023,NaN,NaN,NaN,-0.023,0.547,1,0,57.0,1
573181,FF,0.708647,-0.696363,0.210743,669016,663623,NaN,foul,12.0,Brandon Marsh homers (12) on a fly ball to rig...,R,L,R,S,0,0,-0.590055,1.007694,1.06,2.92,NaN,NaN,NaN,NaN,NaN,NaN,1.240581,-0.682952,-0.342556,-0.721606,0.693715,1.022833,3.19,1.50,211.0,70.6,30.0,94.3,0.284763,1.192326,745539,53.55,NaN,NaN,52,1,4-Seam Fastball,3,4,0.676872,0.0,-0.023,67.0,7.0,NaN,0.023,0.347,3,2,29.0,0


In [12]:
pitch_threshold = 10  # minimum pitches thrown in a season

pitch_data_filtered = pitch_data_filtered.groupby(["pitcher", "pitch_type", "cluster_label"]).size().reset_index(name="count")
pitch_data_filtered = pitch_data_filtered[pitch_data_filtered["count"] > pitch_threshold]

# Find the most common cluster for each pitch type per pitcher
most_common_clusters = pitch_data_filtered.groupby(["pitcher", "pitch_type"])["cluster_label"].agg(lambda x: x.value_counts().idxmax()).reset_index()

pitcher_cluster_df = most_common_clusters.pivot(index="pitcher", columns="pitch_type", values="cluster_label").fillna(-1)
pitcher_cluster_df = pitcher_cluster_df.astype(int)


In [13]:
pitcher_cluster_df.head()

pitch_type,CH,CU,EP,FA,FC,FF,FO,FS,KC,KN,SC,SI,SL,ST,SV
pitcher,,,,,,,,,,,,,,,
434378,1,2,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,0,-1,-1
445276,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,-1,1,0,-1,-1
445926,2,0,-1,-1,0,-1,-1,-1,-1,-1,-1,1,-1,-1,-1
448179,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1
450203,1,0,-1,-1,0,0,-1,-1,-1,-1,-1,1,-1,-1,-1
